<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/Stable_Video_4D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Video 4D on Google Colab (it needs more VRAM!)

I've tested this on A100(40GB), L4 GPU (22GB) but still not working.
Tell me if you find any setting to run on it.

- https://stability.ai/stable-video

- https://ja.stability.ai/blog/stable-video-4d

- https://note.com/aicu/n/n69b846f3a83e


original code and repo
- https://github.com/Stability-AI/generative-models/blob/main/scripts/sampling/simple_video_sample_4d.py


In [ ]:
import os
from google.colab import userdata
# Hugging Face トークンの設定
hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token


# リポジトリのクローン
!git clone https://github.com/Stability-AI/generative-models.git
%cd generative-models

# 依存関係のインストール
!pip install -r ./requirements/pt2.txt

# PyTorchとxFormersの特定バージョンのインストール
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --extra-index-url https://download.pytorch.org/whl/cu118
# ここで再起動が入りますが このセルを再度実行してください。
!pip install xformers==0.0.20




fatal: destination path 'generative-models' already exists and is not an empty directory.
/content/generative-models
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-d38d0u_v/clip_08e8b2026ab7442d911651e47f5e79e1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-d38d0u_v/clip_08e8b2026ab7442d911651e47f5e79e1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 984.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 108.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2

In [ ]:
# プロジェクトのインストール
!pip install -e .

# モデルのダウンロード
%cd /content/generative-models
!mkdir -p outputs
!mkdir -p checkpoints

!wget --header="Authorization: Bearer $HF_TOKEN" -O checkpoints/sv4d.safetensors https://huggingface.co/stabilityai/sv4d/resolve/main/sv4d.safetensors
!wget --header="Authorization: Bearer $HF_TOKEN" -O checkpoints/sv3d_u.safetensors https://huggingface.co/stabilityai/sv3d/resolve/main/sv3d_u.safetensors
!ls -lh checkpoints/*.safetensors

# スクリプトの実行
!python scripts/sampling/simple_video_sample_4d.py --input_path assets/test_video1.mp4 --output_folder outputs/sv4d


Obtaining file:///content/generative-models
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sgm (pyproject.toml) ... done
  Created wheel for sgm: filename=sgm-0.1.0-py3-none-any.whl size=30609 sha256=ce6ba38e4f9c79827e58275880f63274b0e3614db2009943848a8fd8d5db0224
  Stored in directory: /tmp/pip-ephem-wheel-cache-dheqo1yi/wheels/12/9b/27/03142f4dee9fa0a99f6c146eae81eb66e17b781145ecb05fa5
Successfully built sgm
/content/generative-models
--2024-07-25 05:46:00--  https://huggingface.co/stabilityai/sv4d/resolve/main/sv4d.safetensors
Resolving huggingface.co (huggingface.co)... 18.238.109.121, 18.238.109.52, 18.238.109.102, ...
Connecting to huggingface.co (huggingface.co)|18.238.109.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: 

In [ ]:
# モデルダウンロードの確認
%cd /content/generative-models
!ls -lh checkpoints/*.safetensors

# このままだと A100 GPU (40GB)でも動作しない
#!python scripts/sampling/simple_video_sample_4d.py --input_path assets/test_video1.mp4 --output_folder outputs/sv4d

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

#パラメータを小さくしてみましたが動かず
!python scripts/sampling/simple_video_sample_4d.py \
    --input_path assets/test_video2.mp4 \
    --output_folder outputs/sv4d \
    --model_path checkpoints/sv4d.safetensors \
    --model_path_sv3d checkpoints/sv3d_u.safetensors \
    --batch_size 1 \
    --height 32 --width 32 \
    --num_steps 1

# どなたか動作確認していただける方を募集します！


/content/generative-models
-rw-r--r-- 1 root root 8.8G Mar 18 14:59 checkpoints/sv3d_u.safetensors
-rw-r--r-- 1 root root  12G Jul 24 04:38 checkpoints/sv4d.safetensors
Reading assets/test_video2.mp4
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing
Initialized embedder #0: FrozenOpenCLIPImagePredictionEmbedder with 683800065 pa